# creating a single agent that searches linkedin for people - OpenAI

In [96]:
from xpander_sdk import XpanderClient, GraphItem
from openai import OpenAI
from dotenv import load_dotenv
from os import environ

load_dotenv()


True

In [97]:
OPENAI_API_KEY = environ["OPENAI_API_KEY"]
XPANDER_API_KEY = environ["XPANDER_API_KEY"]

## Initialize xpander.ai Client

In [98]:
xpander_client = XpanderClient(api_key=XPANDER_API_KEY)

## Initialize OpenAI Client

In [99]:
openai_client = OpenAI(api_key=OPENAI_API_KEY)

## Create the agent

In [100]:
agent = xpander_client.agents.create(name="Linkedin Researcher")
print(f"created agent {agent.id} - https://app.xpander.ai/agents/{agent.id}")

created agent 619742f9-7c98-4853-a318-f043281973fe - https://app.xpander.ai/agents/619742f9-7c98-4853-a318-f043281973fe


## List interfaces

In [101]:
agentic_interfaces = agent.retrieve_agentic_interfaces()

## Find linkedin agentic interface

In [102]:
linkedin_interface = next((interface for interface in agentic_interfaces if "linkedin" in interface.name.lower()))

## List LinkedIn operations

In [103]:
linkedin_operations = agent.retrieve_agentic_operations(agentic_interface=linkedin_interface)

## Take 2 relevant operations (search profile + get profile)

In [104]:
search_profile_operation = next((operation for operation in linkedin_operations if operation.path == "/search-people"))
get_profile_operation = next((operation for operation in linkedin_operations if operation.path == "/get-profile-data-by-url"))

## Attach the operations to the agent

In [105]:
agent.attach_operations(operations=[search_profile_operation,get_profile_operation])

## Add the operations to the graph

In [106]:
search_profile_node = agent.graph.add_node(GraphItem(agent=agent,item_id=search_profile_operation.id_to_use_on_graph,name=search_profile_operation.name))
get_profile_node = agent.graph.add_node(GraphItem(agent=agent,item_id=get_profile_operation.id_to_use_on_graph,name=get_profile_operation.name))

/Users/morielp/dev/xpander-agents-hub/.venv/lib/python3.12/site-packages/xpander_sdk/__init__.py:215: UserWarning: Typeguard cannot check the IAgentGraphItemSettings protocol because it is a non-runtime protocol. If you would like to type check this protocol, please use @typing.runtime_checkable
  typeguard.check_type(value=value, expected_type=expected_type, collection_check_strategy=typeguard.CollectionCheckStrategy.ALL_ITEMS) # type:ignore


## Connect the graph items

In [107]:
search_profile_node.connect([get_profile_node])

## Sync (deploy)

In [108]:
agent.sync()

## Invoke the agent with query and create an execution task

In [109]:
agent.add_task("""
Find details about David Twizer the founder of xpander.ai
""")

## Init the agent's memory with input (query) and instructions

In [110]:
agent.memory.init_messages(input=agent.execution.input_message,instructions=agent.instructions)

## Run the agent and wait for result

In [111]:
while not agent.is_finished():
    response = openai_client.chat.completions.create(
                model= "gpt-4o",
                messages=agent.messages,
                tools=agent.get_tools(),
                tool_choice=agent.tool_choice,
                temperature=0.0
        )
            
    # add messages directly from the LLM response
    agent.add_messages(response.model_dump())
    
    # extract tool calls from the LLM response
    tool_calls = XpanderClient.extract_tool_calls(llm_response=response.model_dump())
    # run tools
    agent.run_tools(tool_calls=tool_calls)


## Fetch & print the agent execution result

In [112]:
execution_result = agent.retrieve_execution_result()

print("status", execution_result.status)
print("result", execution_result.result)

status ExecutionStatus.COMPLETED
result **David Twizer** is the Co-Founder & CEO of xpander.ai, a company that empowers AI Engineers and AI Leaders to build intelligent AI Agents. He has a rich background in cloud solutions and has previously worked at Amazon Web Services (AWS) in various roles, including Sr. Manager, GenAI Specialist SA, and Principal Solutions Architect.

### Profile Summary:
- **Name:** David Twizer
- **Current Position:** Co-Founder & CEO at xpander.ai
- **Location:** Tel Aviv-Yafo, Tel Aviv District, Israel
- **LinkedIn Profile:** [David Twizer](https://www.linkedin.com/in/dudutwizer)

### Professional Experience:
- **xpander.ai**
  - Co-Founder & CEO (2024 - Present)
  - Focus: Building AI Agents for complex, multi-step actions across systems.

- **Amazon Web Services (AWS)**
  - Sr. Manager, GenAI Specialist SA (Aug 2022 - Dec 2023)
  - Principal Solutions Architect (Sep 2020 - Aug 2022)
  - Specialist Solutions Architect (Oct 2018 - Sep 2020)
  - Technical Acco